# Data cleaning
This script relates to the kickstarter dataset - please run this only after the setup is complete.
It changes column names, drops unnecessary features and checks the currency columns for validity.
Collaborators: Philippa Schindler

In [34]:
import pandas as pd
import re

In [35]:
df = pd.read_csv('data/1_working_data.csv')

## Renaming

In [36]:
# rename blurb to description and goal to goal_orig
df.rename(columns={'blurb':'description','pledged':'pledged_orig','goal':'goal_orig'},inplace=True)

In [37]:
# add prefix date_* to launched_at, created_at, deadline, state_changed_at
df.rename(columns={'launched_at':'date_launch', 'created_at':'date_creation', 'deadline':'date_end', 'state_changed_at':'date_status_change'}, inplace=True)


## Drop

In [38]:
# drop unnecessary columns
df_drop = df.drop(["Unnamed: 0",'currency_symbol','converted_pledged_amount','currency_trailing_code',
    'friends', 'fx_rate','is_backing', 'is_starrable', 'is_starred','permissions','slug','source_url','usd_type'], axis = 1)

In [39]:
#drop duplicate rows & status == fail or successful 
df = df.query('(state == "failed") or (state == "successful")')
df.drop_duplicates(subset = 'id', inplace=True)

In [40]:
# quality check
df.head()

,Unnamed: 0,backers_count,description,converted_pledged_amount,country,date_creation,currency,currency_symbol,currency_trailing_code,current_currency,...,static_usd_rate,usd_pledged,usd_type,creator_id,category_id,category_name,category_slug,category_position,category_parent_id,category_color
1,1,47,A colorful Dia de los Muertos themed oracle de...,1950,US,2017-08-02 14:28:13,USD,$,True,USD,...,1.000000,1950.000000,domestic,723886115,273,Playing Cards,games/playing cards,4,12.0,51627
2,2,271,"Electra's long awaited, eclectic Debut Pop/Roc...",22404,US,2012-09-30 06:45:33,USD,$,True,USD,...,1.000000,22404.000000,international,323849677,43,Rock,music/rock,17,14.0,10878931
3,3,3,The Mist of Tribunal is a turn-based card game...,165,GB,2017-01-07 09:11:11,GBP,£,False,USD,...,1.216066,165.384934,domestic,196281496,273,Playing Cards,games/playing cards,4,12.0,51627
4,4,3,"Livng with a brain impairment, what its like t...",2820,US,2012-12-06 18:04:31,USD,$,True,USD,...,1.000000,2820.000000,domestic,1178460181,48,Nonfiction,publishing/nonfiction,9,18.0,14867664
5,5,35,Annapolis Chamber Players is a non-for profit ...,3725,US,2014-10-24 17:35:50,USD,$,True,USD,...,1.000000,3725.000000,domestic,682189804,36,Classical Music,music/classical music,3,14.0,10878931


## Split category

In [41]:
df[['category_main','category_sub']] = df['category_slug'].str.split('/', expand=True)
df.head()

,Unnamed: 0,backers_count,description,converted_pledged_amount,country,date_creation,currency,currency_symbol,currency_trailing_code,current_currency,...,usd_type,creator_id,category_id,category_name,category_slug,category_position,category_parent_id,category_color,category_main,category_sub
1,1,47,A colorful Dia de los Muertos themed oracle de...,1950,US,2017-08-02 14:28:13,USD,$,True,USD,...,domestic,723886115,273,Playing Cards,games/playing cards,4,12.0,51627,games,playing cards
2,2,271,"Electra's long awaited, eclectic Debut Pop/Roc...",22404,US,2012-09-30 06:45:33,USD,$,True,USD,...,international,323849677,43,Rock,music/rock,17,14.0,10878931,music,rock
3,3,3,The Mist of Tribunal is a turn-based card game...,165,GB,2017-01-07 09:11:11,GBP,£,False,USD,...,domestic,196281496,273,Playing Cards,games/playing cards,4,12.0,51627,games,playing cards
4,4,3,"Livng with a brain impairment, what its like t...",2820,US,2012-12-06 18:04:31,USD,$,True,USD,...,domestic,1178460181,48,Nonfiction,publishing/nonfiction,9,18.0,14867664,publishing,nonfiction
5,5,35,Annapolis Chamber Players is a non-for profit ...,3725,US,2014-10-24 17:35:50,USD,$,True,USD,...,domestic,682189804,36,Classical Music,music/classical music,3,14.0,10878931,music,classical music


In [42]:
# Quality check 
df['category_main'].isna().value_counts()

False    168979
Name: category_main, dtype: int64

In [43]:
df['category_sub'].isna().value_counts()

False    160703
True       8276
Name: category_sub, dtype: int64

In [44]:
# Fill missing sub cats as miscallenous
df['category_sub'].fillna('Misc',inplace=True)

In [45]:
df['category_sub'].isna().value_counts()

False    168979
Name: category_sub, dtype: int64

# Change date to date format

In [46]:
df['date_end'] = pd.to_datetime(df['date_end'])
df['date_launch'] = pd.to_datetime(df['date_launch'])
df['date_status_change'] = pd.to_datetime(df['date_status_change'])
df['date_creation'] = pd.to_datetime(df['date_creation'])

# Add new columns

In [47]:
# length of campaign
df['duration'] = df['date_end'] - df['date_launch']
# quality check
df['duration'].describe()

count                        168979
mean     32 days 15:40:57.190307671
std      11 days 18:11:17.107493517
min                 1 days 00:00:00
25%                29 days 23:00:00
50%                30 days 00:00:00
75%         34 days 03:06:58.500000
max                93 days 02:32:04
Name: duration, dtype: object

In [70]:
# length of description
df['description'] = df['description'].astype('string')
df['description'].dtypes

string[python]

In [67]:
# Length of description + title of project
df['description_length'] = df['description'].apply(len)
# Quality check
df['description_length'].describe()

count    168979.000000
mean        112.865273
std          26.275073
min           1.000000
25%         102.000000
50%         124.000000
75%         132.000000
max         196.000000
Name: description_length, dtype: float64

In [68]:
df['name_length'] = df['name'].apply(len)
# Quality check
df['name_length'].describe()

count    168979.000000
mean         34.804236
std          15.704077
min           1.000000
25%          22.000000
50%          34.000000
75%          49.000000
max          85.000000
Name: name_length, dtype: float64

# Create final csv to use for all further models

In [69]:
#save merged raw dataframe as a file 
df.to_csv('data/2_data.csv')